In [ ]:
'''
Uses the Tensoflow machine learning library to train the Poisson 
autoencoder inverting network (PAIN) to reconstruct compressed 
MNIST images with Poisson noise, and saves the trained model.

Authors: Fabian Santiago
Last Update: August 18, 2024
'''
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
import os
from skimage.util.shape import view_as_windows

In [ ]:
# Set the seed for TensorFlow and Keras
tf.random.set_seed(101)

In [ ]:
# Define sliding window compression
def median_downsampling(in_image, cmp_dim):
    if cmp_dim < 15:
        # Compression dimension to window size
        window_size = 28//cmp_dim
        # Create sliding windows
        windows = view_as_windows(in_image, (window_size, window_size), step=window_size)
        # Calculate the median over each window
        return np.median(windows, axis=(2, 3))
    else:
        return in_image

# Define compression of entries in an array
def down_sample_list(in_array,cmp_dim):
    out_array = np.empty((len(in_array), cmp_dim, cmp_dim))
    
    # Iterate over array
    for idx, image in enumerate(in_array):
        out_array[idx] = median_downsampling(image,cmp_dim)
        
    # Return array containing compressed entries 
    return out_array 

In [ ]:
# Load the MNIST Dataset
########################
# Load only MNIST images
(clean_train, _), (clean_test, _) = tf.keras.datasets.mnist.load_data()

# Normalize data to [0,1]
# clean_train, clean_test = clean_train/255, clean_test/255

# Dimension of the compressed/noisy images (width=height)  
# cmp_dim = 4 (for 4x4), 7, 14, or 28
cmp_dim = 7


# Step 1: Compress images using median in sliding window
cmp_train = down_sample_list(clean_train, cmp_dim)
cmp_test  = down_sample_list(clean_test, cmp_dim)

# Step 2: Add Poisson noise to compressed images
noisy_train = np.clip(np.random.poisson(lam=cmp_train),0,255)
noisy_test  = np.clip(np.random.poisson(lam=cmp_test),0,255)

# Step 3: Reshape Input Arrays
noisy_train = np.array([matrix.reshape(cmp_dim**2,) for matrix in noisy_train/255])
noisy_test  = np.array([matrix.reshape(cmp_dim**2,) for matrix in noisy_test/255])

# Step 4: Reshape Expected Ouput Arraus
clean_train = np.array([matrix.reshape(28**2,) for matrix in clean_train/255])
clean_test  = np.array([matrix.reshape(28**2,) for matrix in clean_test/255])

# noisy_train = 

# # 'clean_train' -> Original 28x28 MNIST images
# # 'noisy_train' -> Compressed/Noisy MNIST images (cmp_dim x cmp_dim)
# MNIST_data  = 'MNIST_data/'+'MNIST_'+str(cmp_dim)+'x'+str(cmp_dim)+'_Training.npz'
# DATA        = np.load(MNIST_data)
# clean_train = DATA['clean_train'][0:50000]
# noisy_train = DATA['noisy_train'][0:50000]
# clean_test  = DATA['clean_train'][50000:]
# noisy_test  = DATA['noisy_train'][50000:]

In [ ]:
noisy_train[0].shape

In [ ]:
# Input Data Values
in_dim   = cmp_dim  # This is the input dimemsions in_dim x in_dim
in_size  = int(in_dim*in_dim)
out_dim  = 28 # never changes because MNIST original are 28 x 28
out_size = int(28*28)

In [ ]:
# Define Poisson autoencoder inverting network (PAIN)
####################################################

encoder_size = 256 # Size of intermediate layer

# Define model architecture
def MODEL(input_shape):
    # Input layer
    inputs = tf.keras.Input(shape=(input_shape,))

    # First Decoder
    Dec1 = tf.keras.layers.Dense(out_size, activation='sigmoid')(inputs)

    # Encoder
    Enc2_hidden = tf.keras.layers.Dense(encoder_size, activation='sigmoid')(Dec1)
    Enc2 = tf.keras.layers.Dense(in_size, activation='sigmoid')(Enc2_hidden)

    # Second Decoder
    Dec2_hidden = tf.keras.layers.Dense(encoder_size, activation='sigmoid')(Enc2)
    outputs = tf.keras.layers.Dense(out_size, activation='sigmoid')(Dec2_hidden)

    # Define the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
# Create the model
PAIN = MODEL(in_size)

# Create an RMSProp optimizer with a specific learning rate
RMSp = tf.keras.optimizers.RMSprop(learning_rate=0.05)

# Compile the model
###################
PAIN.compile(optimizer=RMSp, loss='mean_squared_error')

In [ ]:
PAIN.fit(noisy_train, clean_train, epochs=600, batch_size=250,validation_data=(noisy_test, clean_test))

In [ ]:
# Save Trained Model
####################

# Model directory 
directory = "Trained_Models"

# If directory does not exist, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Save trained model
PAIN.save(f'Trained_Models/PAIN{cmp_dim}x{cmp_dim}.keras')

In [ ]:
# Apply PAIN to all training inputs
pred_train_out = PAIN.predict(noisy_train)

In [ ]:
# Print examples: training set
########################################################

# Create a 3 by 4 subplot handle
fig, axes = plt.subplots(3, 4, figsize=(11, 8))
axes      = axes.flatten()

# Add title
fig.suptitle('Application of PAIN Architecture\n(Training Set)',fontsize=20,fontweight='bold', fontfamily='serif')

# Shift through training dataset 
sft_idx = 90

# Plot noisy and decompressed realizations in each subplot
for idx in range(4):
    # Plot noisy
    axes[idx].imshow(noisy_train[idx+sft_idx].reshape(cmp_dim,cmp_dim),cmap='gray')
    axes[idx].set_xticks([]) # Remove xticklabels
    axes[idx].set_yticks([]) # Remove yticklabels
    axes[idx].set_xlabel('⇩',fontdict={'fontsize': 25, 'fontweight': 'bold', 'fontfamily': 'serif'})
    # Plot decompressed with PAIN
    axes[idx+4].imshow(pred_train_out[idx+sft_idx].reshape(28,28),cmap='gray')
    axes[idx+4].set_xticks([]) # Remove xticklabels
    axes[idx+4].set_yticks([]) # Remove yticklabels
    
    # Plot original
    axes[idx+8].imshow(clean_train[idx+sft_idx].reshape(28,28),cmap='gray')
    axes[idx+8].set_xticks([]) # Remove xticklabels
    axes[idx+8].set_yticks([]) # Remove yticklabels
    
# Set ylables 
axes[0].set_ylabel(f'{cmp_dim} x {cmp_dim}\n(Input)\nCompressed/Noisy',fontdict={'fontsize': 12, 'fontfamily': 'serif'})
axes[4].set_ylabel('28 x 28\n(Output)\nPAIN Reconstruction',fontdict={'fontsize': 12, 'fontfamily': 'serif'})
axes[8].set_ylabel('28 x 28\n(Original)\nMNIST',fontdict={'fontsize': 12, 'fontfamily': 'serif'})

# Adjust layout to decrease padding between subplots
plt.subplots_adjust(wspace=-0.2, hspace=0.25)

# Display the figure
plt.show()
    

In [ ]:
# Apply PAIN to all validation inputs
pred_test_out = PAIN.predict(noisy_test)

In [ ]:
# Print examples: validation set 
################################

# Create a 3 by 4 subplot handle
fig, axes = plt.subplots(3, 4, figsize=(11, 8))
axes      = axes.flatten()

# Add title
fig.suptitle('Application of PAIN Architecture\n(Validation Set)',fontsize=20,fontweight='bold', fontfamily='serif')

# Shift through validation dataset 
sft_idx   = 10

# Plot noisy and decompressed realizations in each subplot
for idx in range(4):
    # Plot noisy
    axes[idx].imshow(noisy_test[idx+sft_idx].reshape(cmp_dim,cmp_dim),cmap='gray')
    axes[idx].set_xticks([]) # Remove xticklabels
    axes[idx].set_yticks([]) # Remove yticklabels
    axes[idx].set_xlabel('⇩',fontdict={'fontsize': 25, 'fontweight': 'bold', 'fontfamily': 'serif'})
    # Plot decompressed with PAIN
    axes[idx+4].imshow(pred_test_out[idx+sft_idx].reshape(28,28),cmap='gray')
    axes[idx+4].set_xticks([]) # Remove xticklabels
    axes[idx+4].set_yticks([]) # Remove yticklabels
    
    # Plot original
    axes[idx+8].imshow(clean_test[idx+sft_idx].reshape(28,28),cmap='gray')
    axes[idx+8].set_xticks([]) # Remove xticklabels
    axes[idx+8].set_yticks([]) # Remove yticklabels
    
# Set ylables 
axes[0].set_ylabel(f'{cmp_dim} x {cmp_dim}\n(Input)\nCompressed/Noisy',fontdict={'fontsize': 12, 'fontfamily': 'serif'})
axes[4].set_ylabel('28 x 28\n(Output)\nPAIN Reconstruction',fontdict={'fontsize': 12, 'fontfamily': 'serif'})
axes[8].set_ylabel('28 x 28\n(Original)\nMNIST',fontdict={'fontsize': 12, 'fontfamily': 'serif'})

# Adjust layout to decrease padding between subplots
plt.subplots_adjust(wspace=-0.2, hspace=0.25)

# Display the figure
plt.show()
    